<a href="https://colab.research.google.com/github/bhavanadevarashetty888/Bhavana_INFO5731_Spring2021/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [10]:
# Write your code here
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import datasets
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

train_data = open("/content/stsa-train.txt","r")

def saperate(ip,ids):
  if(ids == 0):
    return ip[0:1]
  elif(ids == 1):
    return ip[2:len(ip)].replace("\n","")
  pass
  
list1 = train_data.readlines()

train_data_polarities = [int(saperate(i,0)) for i in list1]
train_data = [saperate(i,1) for i in list1]
trainDF = pd.DataFrame(train_data,columns=["sentences"])
trainDF["polarities"] = pd.DataFrame(train_data_polarities)
trainDF.head()

,sentences,polarities
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [13]:
test_data = open("/content/stsa-test.txt","r")
list2 = test_data.readlines()
test_data_polarities = [int(saperate(i,0)) for i in list2]
test_data = [saperate(i,1) for i in list2]
testDF = pd.DataFrame(test_data,columns=["sentences"])
testDF["polarities"] = pd.DataFrame(test_data_polarities)

testDF.head(5)

,sentences,polarities
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [14]:
trainDF.shape

(6920, 2)

In [15]:
testDF.shape

(1821, 2)

In [16]:
trainDF["polarities"].value_counts()

1    3610
0    3310
Name: polarities, dtype: int64

In [17]:
testDF["polarities"].value_counts()

0    912
1    909
Name: polarities, dtype: int64

In [19]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
wordnet= WordNetLemmatizer()
nltk.download('wordnet')
list_of_words = set(stopwords.words('english')) 
import re


def clean(value):
  value = value.lower()
  tokens = value.split(" ")
  value = " ".join(x for x in tokens if x not in string.punctuation)
  value = "".join(i for i in value if i.isalnum() or i == " ")
  tokens = re.split("\W+",value)
  value = [wordnet.lemmatize(x) for x in tokens if x not in list_of_words]
  return value

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
trainDF.head(5)

,sentences,polarities
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [21]:

testDF.head(5)

,sentences,polarities
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=clean_dataset)
tf_idf = vectorizer.fit_transform(trainDF["sentences"])
list_of_column_names = vectorizer.get_feature_names()
tf_idf_df = pd.DataFrame(tf_idf.toarray())
tf_idf_df.columns = list_of_column_names
tf_idf_df.shape

(6920, 13439)

In [29]:
vectorizer1 = TfidfVectorizer(analyzer=clean_dataset,vocabulary=vectorizer.vocabulary_)
vectorizer1.fit(testDF["sentences"])
tf_idf_test = vectorizer1.transform(testDF["sentences"])
op1 = testDF["sentences"]
training_tf_idf_data = vectorizer.transform(trainDF["sentences"])

x_train, x_test, y_train, y_test = train_test_split(tf_idf_df, trainDF['polarities'].values,test_size=0.2)

**Multinomial DB Algorithm**

In [33]:
from sklearn.naive_bayes import MultinomialNB
multinomialNB = MultinomialNB()
multinomialNB_model = multinomialNB.fit(x_train,y_train)
y_op_multinomialNB = multinomialNB_model.predict(x_test)

y_op_multinomialNB

array([1, 1, 0, ..., 1, 1, 1])

In [36]:

from sklearn.metrics import accuracy_score,classification_report
import sys
import pprint
def fetch_the_accuracy(predicted,actual):
  print("\033[1m"+"Accuracy Score = ",round(accuracy_score(predicted,actual),2)*100,"%"+"\033[0m")
  print("\033[1m"+"The Classification report"+"\033[0m")
  print(classification_report(actual,predicted))
  # sys.modules[__name__].__dict__.clear()
  pass
fetch_the_accuracy(y_op_multinomialNB,y_test)

Accuracy Score =  76.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.79      0.68      0.73       669
           1       0.73      0.84      0.78       715

    accuracy                           0.76      1384
   macro avg       0.76      0.76      0.76      1384
weighted avg       0.76      0.76      0.76      1384



In [40]:
def analyse_the_accuracy(n_splits):
  # evaluate the algorythm
  # fix the random seed
  # must use the same seed value so that the same subsets can be obtained
  # for each time the process is repeated
  seed = 7
  # split the whole dataset into folds 
  kfold = KFold(n_splits, random_state=seed,shuffle=True)
  # for logistic regression, we can use the accuracy level to evaluate the model / algorithm
  scoring = 'accuracy'
  return kfold
  pass

In [41]:
from sklearn.model_selection import cross_val_score
def cross_val_score_computation(modelObject, x_test,y_test):
  kfold = analyse_the_accuracy(10)
  print("\033[1m"+"Cross Validation Score",round(cross_val_score(modelObject,x_test,y_test,cv=kfold).mean()*100,2),"%","\033[0m")
  pass
cross_val_score_computation(multinomialNB_model,x_test,y_test)

Cross Validation Score 70.6 % 


**SVM**

In [44]:
from sklearn.svm import LinearSVC
linearSVC = LinearSVC()
linearSVC_model = linearSVC.fit(x_train,y_train)
y_op_linearSVC = linearSVC_model.predict(x_test)
y_op_test_linearSVC = linearSVC_model.predict(tf_idf_test)
y_op_test_linearSVC

array([0, 0, 1, ..., 1, 0, 1])

In [45]:
fetch_the_accuracy(y_op_linearSVC,y_test)

Accuracy Score =  76.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.76      0.74      0.75       669
           1       0.76      0.78      0.77       715

    accuracy                           0.76      1384
   macro avg       0.76      0.76      0.76      1384
weighted avg       0.76      0.76      0.76      1384



In [48]:

cross_val_score_computation(linearSVC_model,x_test,y_test)

Cross Validation Score 71.02 % 


In [49]:
fetch_the_accuracy(y_op_test_linearSVC,testDF["polarities"].values)

Accuracy Score =  79.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.81      0.77      0.79       912
           1       0.78      0.82      0.80       909

    accuracy                           0.79      1821
   macro avg       0.79      0.79      0.79      1821
weighted avg       0.79      0.79      0.79      1821



In [50]:
cross_val_score_computation(linearSVC_model,tf_idf_test,y_op_test_linearSVC)

Cross Validation Score 79.02 % 


**KNN Algorithm**

In [55]:
from sklearn.neighbors import KNeighborsClassifier
kneighborsClassifier = KNeighborsClassifier(n_neighbors=10,algorithm='brute')
kneighborsClassifier_model = kneighborsClassifier.fit(x_train,y_train)

y_op_kneighborsClassifier = kneighborsClassifier_model.predict(x_test)
y_op_test_kneighborsClassifier = kneighborsClassifier_model.predict(tf_idf_test)
y_op_test_kneighborsClassifier

array([0, 1, 0, ..., 0, 0, 0])

In [56]:
fetch_the_accuracy(y_op_kneighborsClassifier,y_test)

Accuracy Score =  68.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.62      0.89      0.73       669
           1       0.83      0.49      0.61       715

    accuracy                           0.68      1384
   macro avg       0.72      0.69      0.67      1384
weighted avg       0.73      0.68      0.67      1384



In [57]:
cross_val_score_computation(kneighborsClassifier_model,x_test,y_test)

Cross Validation Score 65.53 % 


In [58]:
fetch_the_accuracy(y_op_test_kneighborsClassifier,testDF["polarities"].values)

Accuracy Score =  68.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.63      0.88      0.73       912
           1       0.80      0.48      0.60       909

    accuracy                           0.68      1821
   macro avg       0.71      0.68      0.66      1821
weighted avg       0.71      0.68      0.66      1821



In [59]:
cross_val_score_computation(kneighborsClassifier_model,tf_idf_test,y_op_test_kneighborsClassifier)

Cross Validation Score 71.61 % 


**Decision Tree Algorithm**

In [63]:
from sklearn.tree import DecisionTreeClassifier
decisionTreeClassifier = DecisionTreeClassifier()
decisionTreeClassifier_model = decisionTreeClassifier.fit(x_train,y_train)

y_op_decisionTreeClassifier = decisionTreeClassifier_model.predict(x_test)


In [65]:
y_op_test_decisionTreeClassifier = decisionTreeClassifier_model.predict(tf_idf_test)

y_op_test_decisionTreeClassifier

array([0, 0, 0, ..., 1, 1, 0])

In [66]:
fetch_the_accuracy(y_op_decisionTreeClassifier,y_test)

Accuracy Score =  65.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.63      0.68      0.66       669
           1       0.68      0.62      0.65       715

    accuracy                           0.65      1384
   macro avg       0.65      0.65      0.65      1384
weighted avg       0.65      0.65      0.65      1384



In [67]:
cross_val_score_computation(decisionTreeClassifier_model,x_test,y_test)

Cross Validation Score 56.21 % 


In [68]:
fetch_the_accuracy(y_op_test_decisionTreeClassifier,testDF["polarities"].values)

Accuracy Score =  65.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.65      0.68      0.66       912
           1       0.66      0.62      0.64       909

    accuracy                           0.65      1821
   macro avg       0.65      0.65      0.65      1821
weighted avg       0.65      0.65      0.65      1821



In [69]:
cross_val_score_computation(decisionTreeClassifier_model,tf_idf_test,y_op_test_decisionTreeClassifier)

Cross Validation Score 69.96 % 


**Random Forest Algorithm**

In [71]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier = RandomForestClassifier()
randomForestClassifier_model = randomForestClassifier.fit(x_train,y_train)

In [77]:
y_op_randomForestClassifier = randomForestClassifier_model.predict(x_test)

In [75]:

y_op_test_randomForestClassifier = randomForestClassifier_model.predict(tf_idf_test)

In [78]:
fetch_the_accuracy(y_op_randomForestClassifier,y_test)

Accuracy Score =  72.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.71      0.70      0.70       669
           1       0.72      0.73      0.73       715

    accuracy                           0.72      1384
   macro avg       0.71      0.71      0.71      1384
weighted avg       0.72      0.72      0.72      1384



In [79]:
cross_val_score_computation(randomForestClassifier_model,x_test,y_test)

Cross Validation Score 62.5 % 


In [81]:
fetch_the_accuracy(y_op_test_randomForestClassifier,testDF["polarities"].values)

Accuracy Score =  73.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.73      0.75      0.74       912
           1       0.74      0.72      0.73       909

    accuracy                           0.73      1821
   macro avg       0.73      0.73      0.73      1821
weighted avg       0.73      0.73      0.73      1821



In [82]:
cross_val_score_computation(randomForestClassifier_model,tf_idf_test,y_op_test_randomForestClassifier)

Cross Validation Score 77.65 % 


**XGBoost Algorithm**

In [85]:
from xgboost import XGBClassifier
xgbClassifier = XGBClassifier()
xgbClassifier_model = xgbClassifier.fit(x_train,y_train)



In [86]:
y_op_xgbClassifier= xgbClassifier_model.predict(x_test)

In [87]:
op1 = pd.DataFrame(tf_idf_test.toarray())
op1.columns = vectorizer1.get_feature_names()

In [88]:
y_op_test_xgbClassifier= xgbClassifier_model.predict(op1)

In [89]:
fetch_the_accuracy(y_op_xgbClassifier,y_test)

Accuracy Score =  64.0 %
The Classification report
              precision    recall  f1-score   support

           0       0.72      0.41      0.52       669
           1       0.60      0.85      0.71       715

    accuracy                           0.64      1384
   macro avg       0.66      0.63      0.61      1384
weighted avg       0.66      0.64      0.62      1384



In [90]:
cross_val_score_computation(xgbClassifier_model,x_test,y_test)

Cross Validation Score 61.56 % 
